In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Paths and stuff
import os
import sys

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [2]:
ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_13point5CS_spinup_0ka/grl40_bm5_paleo17a_CT4_BH0_13point5CS_spinup_0ka0006.nc')
ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_final = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp/grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp0006.nc')
H_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/bm5_data_40kms.nc")
age_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/age_data_40kms.nc")

H_data = H_data_40["H"].data
H_uncert_data = H_data_40["H_uncert"].data
age_c_data = age_data_40["age_c"].data
age_c_uncert_real_data = age_data_40["age_c_uncert_real"].data

mask_H = H_data >= 0.0
mask_H = mask_H.astype(float)
mask_age_c = np.zeros(age_c_data.shape)

for kc in range(age_c_data.shape[0]):
    for j in range(age_c_data.shape[1]):
        for i in range(age_c_data.shape[2]):
            if age_c_uncert_real_data[kc, j, i] > 0 and age_c_data[kc, j, i] >= 0 and age_c_data[kc, j, i] <= 60000 and H_data[j, i] >= 2000.0:
                mask_age_c[kc, j, i] = 1.0

In [3]:
sicopolis_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_tuneAll'
simulation = 'grl40_bm5_paleo17a_CT4_BH0_AC_BM5_m11ka_pkp'
dict_sico_out_folder_prefixes = {"nodiff": "N",
                                 "tlm": "FORWARD",
                                 "adj": "ADJOINT",
                                 "tlm_action": "FORWARDHESSACTION",
                                 "adj_action": "ADJHESSACTION"}
dict_ad_exec_cmds_suffixes = {"nodiff": "nodiff",
                              "tlm": "forward",
                              "adj": "adjoint",
                              "tlm_action": "forwardhessaction",
                              "adj_action": "adjointhessaction"}
dict_ad_log_file_suffixes = {"nodiff": "nodiff",
                             "tlm": "tlm",
                             "adj": "adj",
                             "tlm_action": "tlm_hessaction",
                             "adj_action": "adj_hessaction"}
dict_ad_nc_suffixes = {"nodiff": "nodiff",
                       "tlm": "tlm",
                       "adj": "adj",
                       "tlm_action": "tlm_hessaction",
                       "adj_action": "adj_hessaction"}

KCMAX = 80
exp_sigma_level = dataCleaner.exp_sigma_level(zeta = np.arange(0,1+1./KCMAX,1./KCMAX),
                                              exponent = 2.0)
xModel40       = np.arange(-72.,97.,4.0)*10
yModel40       = np.arange(-345.,-56.,4.0)*10
time_ad = np.arange(12, dtype=float)
IMAX = xModel40.shape[0]-1
JMAX = yModel40.shape[0]-1

data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['q_geo'].data
log_q_geo = np.where(data > 0, np.log10(data), -100)
data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['H'].data
log_H = np.where(data > 0, np.log10(data), -100)

log_c_slide_init = np.log10(13.5)*np.ones((JMAX+1, IMAX+1))
log_gamma_s = np.log10(0.070458)*np.ones((JMAX+1, IMAX+1))
log_s_stat = np.log10(5.0)
log_beta1 = np.log10(2.73)
log_beta2 = np.log10(7.28)
log_Pmax = np.log10(0.6)
log_mu = np.log10(9.7155)
log_c_dis_da = np.log10(15659.0)

temp_c = np.zeros((KCMAX+1, JMAX+1, IMAX+1), dtype=float)
data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['age_c'].data
log_age_c = np.where(data > 0, np.log10(data), -100)

dict_og_state_fields_vals = {"xx_c_slide_init": log_c_slide_init,
                    "xx_q_geo": log_q_geo,
                    "xx_H": log_H,
                    "xx_gamma_s": log_gamma_s,
                    "xx_s_stat": log_s_stat,
                    "xx_beta1": log_beta1,
                    "xx_beta2": log_beta2,
                    "xx_Pmax": log_Pmax,
                    "xx_mu": log_mu,
                    "xx_c_dis_da": log_c_dis_da,
                    "xx_temp_c": temp_c,
                    "xx_age_c": log_age_c}

dict_state_fields_num_dims = {"xx_c_slide_init": "2D",
                        "xx_q_geo": "2D",
                        "xx_H": "2D",
                        "xx_gamma_s": "2D",
                        "xx_s_stat": "2D",
                        "xx_beta1": "2D",
                        "xx_beta2": "2D",
                        "xx_Pmax": "2D",
                        "xx_mu": "2D",
                        "xx_c_dis_da": "2D",
                        "xx_temp_c": "3D",
                        "xx_age_c": "3D"}

dict_state_coords = {"time_ad": time_ad,
               "zeta_c": exp_sigma_level,
               "y": yModel40,
               "x": xModel40}

dict_state_attrs_type = {"xx_c_slide_init": "nodiff",
                   "xx_q_geo": "nodiff",
                   "xx_H": "nodiff",
                   "xx_gamma_s": "nodiff",
                   "xx_s_stat": "nodiff",
                   "xx_beta1": "nodiff",
                   "xx_beta2": "nodiff",
                   "xx_Pmax": "nodiff",
                   "xx_mu": "nodiff",
                   "xx_c_dis_da": "nodiff",
                   "xx_temp_c": "nodiff",
                   "xx_age_c": "nodiff"}

dict_state_fields_or_scalars = {"xx_c_slide_init": "field",
                          "xx_q_geo": "field",
                          "xx_H": "field",
                          "xx_gamma_s": "field",
                          "xx_s_stat": "scalar",
                          "xx_beta1": "scalar",
                          "xx_beta2": "scalar",
                          "xx_Pmax": "scalar",
                          "xx_mu": "scalar",
                          "xx_c_dis_da": "scalar",
                          "xx_temp_c": "field",
                          "xx_age_c": "field"}

dict_masks_observables = {"H": mask_H*H_uncert_data**(-2)}

/tmp/ipykernel_380953/294431682.py:36: RuntimeWarning: divide by zero encountered in log10
  log_H = np.where(data > 0, np.log10(data), -100)
/tmp/ipykernel_380953/294431682.py:49: RuntimeWarning: divide by zero encountered in log10
  log_age_c = np.where(data > 0, np.log10(data), -100)


In [4]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

DA.gradient_descent(2, init_alpha = 1.e-6)

-------------------------------------
iter 0, fc = 2459227.2817783635
-------------------------------------
ratio = 0.27018780340725473, alpha = 1e-06
-------------------------------------
iter 1, fc = 917817.9587995579
-------------------------------------
ratio = 0.040379157057724664, alpha = 1e-06
-------------------------------------
iter 2, fc = 687456.6754333116
-------------------------------------


In [7]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_s_stat", "xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

fc = DA.eval_cost()

ds_x = DA.create_ad_tlm_action_input_nc(bool_randomize = True)
ds_x_tlm_only = DA.subset_of_ds(ds_x, "type", "tlm")
    
ds_Ax = DA.eval_tlm_action()
ds_noise_cov_inv_Ax = DA.eval_noise_cov_inv_action(ds_Ax)
ds_H_misfit_x = DA.eval_misfit_hessian_action()

DA.l2_inner_product([ds_x_tlm_only, ds_H_misfit_x], ["tlm", "adj"]), DA.l2_inner_product([ds_Ax, ds_noise_cov_inv_Ax], ["tlmhessaction", "adjhessaction"])

(43269317.652963266, 43269317.65296327)

In [9]:
DA = optim_new.DataAssimilation(sicopolis_dir, simulation,
                                dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                                dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                                dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars, dict_masks_observables,
                                ["xx_s_stat", "xx_c_slide_init"])

ds_inp_nodiff = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                    dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                    "nodiff")
ds_inp_adj = DA.create_ad_nodiff_or_adj_input_nc(dict_og_state_fields_vals, dict_state_fields_num_dims, 
                                                 dict_state_coords, dict_state_attrs_type, dict_state_fields_or_scalars,
                                                 "adj")

fc = DA.eval_cost()

dict_tlm_action_only_fields_vals = {}
for var in ds_x_tlm_only:
    if DA.dict_tlm_action_fields_or_scalars[var] == "scalar":
        dict_tlm_action_only_fields_vals[var] = ds_x_tlm_only[var].data[0].copy()
    else:
        dict_tlm_action_only_fields_vals[var] = ds_x_tlm_only[var].data.copy()
ds_x = DA.create_ad_tlm_action_input_nc(dict_tlm_action_only_fields_vals)
    
ds_Ax = DA.eval_tlm_action()
ds_noise_cov_inv_Ax = DA.eval_noise_cov_inv_action(ds_Ax)
ds_H_misfit_x = DA.eval_misfit_hessian_action()

DA.l2_inner_product([ds_x_tlm_only, ds_H_misfit_x], ["tlm", "adj"]), DA.l2_inner_product([ds_Ax, ds_noise_cov_inv_Ax], ["tlmhessaction", "adjhessaction"])

(43269317.652963266, 43269317.65296327)